In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14289,2024-03-20,Eua Nba,20:00,Cleveland Cavaliers,Miami Heat,1.51,2.33,202.5,1.83,1.97,-4.5,1.98,2.68,MapS6sQ7,0.662252,0.429185,0.546448,0.507614,0.091436,0.6,0.2,NaN,NaN,187.588,27.362773,0.145866,282.564,79.934328,0.282889,176.860,334.718,200.91,290.52,0.0,0.0,0.0,0.0,0.301994,0.052103,0.212436,-0.29,-0.058,-8.793103,0.618051,0.7,0.081949,6.88,1.376,0.966570,0.429239,0.7,0.270761
14290,2024-03-20,Europa Euroliga,14:45,Fenerbahce,Barcelona,1.60,2.57,163.5,1.85,2.09,-3.5,1.88,3.20,EZzRVu1q,0.625000,0.389105,0.540541,0.478469,0.014105,0.8,0.4,NaN,NaN,149.454,31.558149,0.211156,124.532,25.941767,0.208314,137.928,148.880,146.52,152.24,0.0,0.0,0.0,0.0,0.328966,0.086145,0.367473,1.94,0.388,1.546392,0.677597,0.9,0.222403,-1.47,-0.294,-5.340136,0.586995,0.4,-0.186995
14291,2024-03-20,Europa Euroliga,16:00,Crvena zvezda,Panathinaikos,2.52,1.56,158.5,1.91,1.91,3.5,1.95,1.31,A1UZTJVe,0.396825,0.641026,0.523560,0.523560,0.037851,0.4,0.4,NaN,NaN,198.198,99.399075,0.501514,165.156,91.424044,0.553562,137.582,179.776,144.00,148.20,0.0,0.0,0.0,0.0,0.332756,0.000000,0.277637,-3.60,-0.720,-2.111111,0.599664,0.4,-0.199664,0.32,0.064,8.750000,0.516606,0.5,-0.016606
14292,2024-03-20,Europa Euroliga,16:30,Valencia,Bayern,1.61,2.42,155.5,1.83,1.95,-4.5,2.00,3.40,lhTwTwo2,0.621118,0.413223,0.546448,0.512821,0.034341,1.0,0.4,NaN,NaN,166.510,34.824964,0.209146,173.090,80.180276,0.463229,142.378,235.432,170.04,227.04,0.0,0.0,0.0,0.0,0.284246,0.044896,0.366648,-1.82,-0.364,-1.675824,0.586126,0.5,-0.086126,0.24,0.048,29.583333,0.347813,0.3,-0.047813
14293,2024-03-20,Argentina Liga A,20:10,Ferro,San Lorenzo,1.94,1.78,152.5,1.87,1.83,1.5,1.80,1.65,Uoi2emB0,0.515464,0.561798,0.534759,0.546448,0.077262,0.4,0.4,NaN,NaN,176.082,63.424759,0.360200,214.764,96.908793,0.451234,140.660,219.842,111.36,103.95,0.0,0.0,0.0,0.0,0.060826,0.015289,0.061488,-0.05,-0.010,-94.000000,0.000000,0.0,0.000000,-1.02,-0.204,-3.823529,0.000000,0.0,0.000000
14294,2024-03-20,Bósnia E Herzegovina Prvenstvo Bih,15:00,Mrkonjic Grad,Spars Ilidza,1.69,2.06,166.5,1.81,1.85,-2.5,1.94,2.50,hpnL4cV8,0.591716,0.485437,0.552486,0.540541,0.077153,0.2,0.4,NaN,NaN,301.300,242.200003,0.803850,207.256,90.855135,0.438372,170.678,216.606,102.75,197.88,0.0,0.0,0.0,0.0,0.139536,0.015456,0.178369,1.24,0.248,2.782258,0.000000,0.0,0.000000,-0.75,-0.150,-7.066667,0.000000,0.0,0.000000
14295,2024-03-20,China Cba,08:35,Beijing,Shanghai,1.78,1.97,197.5,1.81,1.89,-1.5,1.90,2.13,ADQM9yCo,0.561798,0.507614,0.552486,0.529101,0.069412,0.2,0.6,NaN,NaN,265.476,84.261181,0.317397,221.774,136.201192,0.614144,206.008,243.156,236.90,194.48,0.0,0.0,0.0,0.0,0.071653,0.030578,0.080712,-0.75,-0.150,-5.200000,0.616264,0.5,-0.116264,4.14,0.828,1.171498,0.532071,0.6,0.067929
14296,2024-03-20,Eslováquia Extraliga,14:00,BKM Lucenec,Handlova,2.02,1.72,166.5,1.87,1.79,1.5,1.85,1.61,zPuHOZbC,0.495050,0.581395,0.534759,0.558659,0.076445,0.6,0.2,NaN,NaN,324.648,262.599048,0.808873,153.592,9.582135,0.062387,177.252,184.560,174.84,162.52,0.0,0.0,0.0,0.0,0.113440,0.030912,0.098096,-1.25,-0.250,-4.080000,0.523227,0.7,0.176773,-0.02,-0.004,-180.000000,0.000000,0.0,0.000000
14297,2024-03-20,Eua Ncaa,19:40,Montana State,Grambling St.,1.61,2.42,134.5,1.91,1.91,-4.5,2.00,3.45,6mO9dX5s,0.621118,0.413223,0.523560,0.523560,0.034341,0.2,0.6,NaN,NaN,143.560,53.338805,0.371544,125.872,48.408600,0.384586,125.362,158.688,165.68,179.76,0.0,0.0,0.0,0.0,0.284246,0.000000,0.376259,1.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Cleveland Cavaliers,Miami Heat,202.5,1.83,1.0000,Over
1,14:45,Europa Euroliga,Fenerbahce,Barcelona,163.5,1.85,1.0000,Over
2,16:30,Europa Euroliga,Valencia,Bayern,155.5,1.83,1.0000,Over
3,15:00,Bósnia E Herzegovina Prvenstvo Bih,Mrkonjic Grad,Spars Ilidza,166.5,1.81,1.0000,Over
4,08:35,China Cba,Beijing,Shanghai,197.5,1.81,0.9977,Over
5,19:40,Eua Ncaa,Montana State,Grambling St.,134.5,1.91,1.0000,Over
6,20:00,Eua Nit,Seton Hall,Saint Josephs,145.5,1.91,1.0000,Over
7,21:00,Eua Nit,Princeton,UNLV,138.5,1.91,1.0000,Over
8,21:00,Eua Nit,Wake Forest,Appalachian State,145.5,1.91,1.0000,Over
9,16:00,Europa Liga Dos Campeões,Ludwigsburg,Bonn,166.5,1.89,1.0000,Over
